In [1]:
import csv
import numpy as np
from numpy import *
import pandas as pd

In [2]:
emails_data = pd.read_csv('file:///Users/sultangalymjanuly/Desktop/PythonFINAL/emails.csv')
print(emails_data.shape)
emails_data

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...


In [14]:
import PyPDF2 
  
# creating a pdf file object 
pdfFileObj = open('/Users/sultangalymjanuly/Desktop/PythonFINAL/enron61702insiderpay.pdf', 'rb') 
  
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

In [15]:
print(pdfReader.numPages)

10


In [16]:
pageObj = pdfReader.getPage(0) 

In [17]:
print(pageObj.extractText())

In re:   Enron Corp.Case No. 01-16034Payments to InsidersSummary Schedule of all Debtors CombinedEXHIBIT 3b.2PaymentsStock Value  (13)InsiderSalary  (1)Bonus  (2)Long Term Incentive  (3)Deferred Income  (4)Deferral Payments  (5)Loan Advances  (6)Other  (7)Expenses  (8)Director Fees  (9)Total PaymentsExercised Stock Options  (10)Restricted Stock  (11)Restricted Stock Deferred  (12)Total Stock ValueALLEN, PHILLIP K$201,955$4,175,000$304,805($3,081,055)$2,869,717- $152$13,868- $4,484,442$1,729,541$126,027($126,027)$1,729,541BADUM, JAMES P-                        -                        -                        -                        178,980             -                        -                        3,486                 -                        182,466               257,817             -                          -                          257,817               BANNANTINE, JAMES M477                    -                        -                        (5,104)               -         